In [1]:
%pip install torch transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import random
import time

def sample_random(p: torch.Tensor) -> torch.Tensor:
    """Samples a token index from the given probability distribution p."""
    return torch.multinomial(p, num_samples=1)

def relu_n(x: torch.Tensor) -> torch.Tensor:
    """Computes the ReLU function and applies normalization"""
    x_relu = torch.relu(x)
    return x_relu / torch.sum(x_relu)

def predict(model: AutoModelForCausalLM, x: torch.Tensor) -> torch.Tensor:
    """
    Gets the probability distribution over the next token given the input sequence x using the model.
        - model: the language model to use for prediction
        - x: the input sequence (token IDs)
    Returns the probability distribution over the next token.
    """
    # Get logits from the model
    with torch.no_grad():
        outputs = model(x)
        logits = outputs.logits

    # Sample next token from the last position
    next_token_logits = logits[:, -1, :]
    return torch.softmax(next_token_logits, dim=-1)

def auto_regressive_sampling(model: AutoModelForCausalLM, x: torch.Tensor, T: int) -> torch.Tensor:
    """
    Auto-regressive sampling from the target model for T steps.
        - model: the target language model to sample from
        - x: the initial input sequence (token IDs)
        - T: the number of tokens to sample
    Returns the generated sequence of token IDs after T steps.
    """
    for _ in range(T):
        # get probabilities
        probabilities = predict(model, x)

        # sample next token from the target model
        predicted_token = sample_random(probabilities)

        # Concatenate the new token
        x = torch.cat([x, predicted_token], dim=-1)

    return x

def speculative_sampling(
        target_model: AutoModelForCausalLM,
        draft_model: AutoModelForCausalLM,
        x: torch.Tensor,
        K: int,
        T: int,
        eps=1e-10
    ) -> torch.Tensor:
    """
    Speculative sampling from the target model using a draft model for K steps.
        - target_model: the target language model to sample from
        - draft_model: the draft language model used for proposing tokens
        - x: the initial input sequence (token IDs)
        - K: the number of speculative steps to perform
        - T: the total number of tokens to sample (including speculative steps)
    Returns the generated sequence of token IDs after T steps.
    """
    n = x.shape[-1]
    T += n

    while n < T:
        # Save the starting position for this iteration
        n_start = n

        # Drafting: auto-regressive sampling using the draft model
        x_draft = x.clone()
        draft_probs = []

        for _ in range(K):
            # get probabilities
            p = predict(draft_model, x_draft)
            draft_probs.append(p)

            # sample next token from the draft model
            predicted_token = sample_random(p)

            # Concatenate the new token
            x_draft = torch.cat([x_draft, predicted_token], dim=-1)

        # Verification: get target model probabilities for entire draft sequence
        with torch.no_grad():
            outputs = target_model(x_draft)
            logits = outputs.logits

        target_probs = []
        for k in range(K + 1):
            pos = n_start - 1 + k
            next_token_logits = logits[:, pos, :]
            target_probs.append(torch.softmax(next_token_logits, dim=-1))

        # Correction: accept or reject predicted tokens
        all_accepted = True
        for k in range(K):
            j = x_draft[:, n_start + k]  # Token at position n_start+k

            p_j = draft_probs[k][0, j.item()]  # Draft probability for token j
            q_j = target_probs[k][0, j.item()]  # Target probability for token j

            r = random.random()
            if r < min(1.0, (q_j / (p_j + eps)).item()):
                # token accepted
                x = torch.cat([x, j.unsqueeze(0)], dim=-1)
                n += 1
            else:
                # token rejected, resample
                adjusted_probs = relu_n(target_probs[k][0] - draft_probs[k][0])
                resampled_token = sample_random(adjusted_probs.unsqueeze(0))
                x = torch.cat([x, resampled_token], dim=-1)
                n += 1
                all_accepted = False
                break

        if all_accepted:
            # sample an extra token from target model at the last position
            x = torch.cat([x, sample_random(target_probs[-1])], dim=-1)
            n += 1

    return x

def decode(output_ids: torch.Tensor, tokenizer: AutoTokenizer) -> str:
    """
    Decodes the generated token IDs into text using the model's tokenizer.
        - output_ids: the sequence of generated token IDs
        - tokenizer: the tokenizer used for decoding
    Returns the decoded text string.
    """
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [14]:
def speculative_sampling_batched(
        target_model: AutoModelForCausalLM,
        draft_model: AutoModelForCausalLM,
        x: torch.Tensor,
        K: int,
        T: int,
        eps=1e-10
    ) -> torch.Tensor:
    """Optimized speculative sampling with batched draft generation."""
    n = x.shape[-1]
    T += n
    device = x.device

    while n < T:
        n_start = n

        # Drafting: Generate K tokens in a more efficient way
        x_draft = x.clone()
        draft_probs = []

        # Still need sequential generation but minimize overhead
        with torch.no_grad():
            for _ in range(K):
                outputs = draft_model(x_draft)
                logits = outputs.logits[:, -1, :]
                p = torch.softmax(logits, dim=-1)
                draft_probs.append(p)

                predicted_token = torch.multinomial(p, num_samples=1)
                x_draft = torch.cat([x_draft, predicted_token], dim=-1)

        # Verification: Single forward pass for all K tokens
        with torch.no_grad():
            outputs = target_model(x_draft)
            logits = outputs.logits

        # Compute all target probs at once
        target_probs = torch.softmax(logits[:, n_start-1:n_start+K, :], dim=-1)

        # Correction: vectorized acceptance check
        all_accepted = True
        for k in range(K):
            j = x_draft[:, n_start + k]

            p_j = draft_probs[k][0, j.item()]
            q_j = target_probs[0, k, j.item()]

            r = torch.rand(1, device=device).item()
            if r < min(1.0, (q_j / (p_j + eps)).item()):
                x = torch.cat([x, j.unsqueeze(0)], dim=-1)
                n += 1
            else:
                adjusted_probs = relu_n(target_probs[0, k] - draft_probs[k][0])
                resampled_token = sample_random(adjusted_probs.unsqueeze(0))
                x = torch.cat([x, resampled_token], dim=-1)
                n += 1
                all_accepted = False
                break

        if all_accepted:
            bonus_probs = torch.softmax(logits[:, n_start + K - 1, :], dim=-1)
            x = torch.cat([x, sample_random(bonus_probs)], dim=-1)
            n += 1

    return x

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [26]:
target_model_name = "gpt2-large"
target_model = torch.compile(AutoModelForCausalLM.from_pretrained(target_model_name).to(device).eval())
tokenizer = AutoTokenizer.from_pretrained(target_model_name)

input_seq = "Once upon a time"
input_ids = tokenizer.encode(input_seq, return_tensors="pt").to(device)

print(f"\nRunning auto-regressive sampling using {target_model_name}")
start = time.perf_counter()

output_ids = auto_regressive_sampling(target_model, input_ids, T=100)
text = decode(output_ids, tokenizer)

elapsed_time = time.perf_counter() - start

print(f"Output: {text}")
print(f"Time: {elapsed_time:.2f}s")

Loading weights:   0%|          | 0/436 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2-large
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...35}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Running auto-regressive sampling using gpt2-large
Output: Once upon a time the programme belonged to Company I, the Indian Infantry Guards, who lost all means of official adaptation. Efforts were made to back it up in light of scientific knowledge, knowledge acquired by mail from foreign officers, published publications in French, German and English, modern legislation with local regulations, American propensities keeping Group Kempe out of plain sight, and eight Christian Brothers fighting numerically the number of fifty-eight men required to have any chance of action, not counting the twelve National Guardsmen expected
Time: 2.26s


In [27]:
# Speculative sampling
draft_model_name = "gpt2"
draft_model = torch.compile(AutoModelForCausalLM.from_pretrained(draft_model_name).to(device).eval())

draft_model.config.use_cache = True
target_model.config.use_cache = True

input_seq = "Once upon a time"
input_ids = tokenizer.encode(input_seq, return_tensors="pt").to(device)

print(f"\nRunning speculative sampling using {target_model_name} with draft model {draft_model_name}")
start = time.perf_counter()

output_ids = speculative_sampling_batched(target_model, draft_model, input_ids, K=5, T=100)
text = decode(output_ids, tokenizer)

elapsed_time = time.perf_counter() - start

print(f"Output: {text}")
print(f"Time: {elapsed_time:.2f}s")

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Running speculative sampling using gpt2-large with draft model gpt2
Output: Once upon a time I would have been frozen with fear under the sawdust-choked roadside." (See Every minute leads to annihilation.)

Living in upstate New York

Humans in their Christian-dominated world generally see any type of food, dress, song or dive as forbidden, upon pain of hellfire and eternal damnation.

However, on a recent visit to upstate New York (8750 Queen Anne Avenue), my daughter and I had dinner at a ranch owned by a man
Time: 1.61s
